In [1]:
# .whlファイルをローカルのinputからインストールする
!pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
!pip install /kaggle/input/pip-i!pip install /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
!pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/pytorch_lightning-2.4.0-py3-none-any.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/torchmetrics-1.5.2-py3-none-any.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/pytorch_tabnet-4.1.0-py3-none-any.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/einops-0.7.0-py3-none-any.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/pytorch_tabular-1.1.1-py2.py3-none-any.whlnstall-lifelines/autograd-gamma-0.5.0.tar.gz
!pip install /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/formulaic-1.0.2-py3-none-any.whl
!pip install /kaggle/input/pip-install-lifelines/lifelines-0.30.0-py3-none-any.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/pytorch_lightning-2.4.0-py3-none-any.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/scikit_learn-1.6.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/torchmetrics-1.5.2-py3-none-any.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/pytorch_tabnet-4.1.0-py3-none-any.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/einops-0.7.0-py3-none-any.whl
!pip install /kaggle/input/download-lightning-and-pytorch-tabular/pytorch_tabular-1.1.1-py2.py3-none-any.whl

Processing /kaggle/input/pip-install-lifelines/autograd-1.7.0-py3-none-any.whl
autograd is already installed with the same version as the provided wheel. Use --force-reinstall to force an installation of the wheel.
ERROR: Invalid requirement: '/kaggle/input/pip-i!pip': Expected package name at the start of dependency specifier
    /kaggle/input/pip-i!pip
    ^
Hint: It looks like a path. File '/kaggle/input/pip-i!pip' does not exist.
Processing /kaggle/input/pip-install-lifelines/autograd-gamma-0.5.0.tar.gz
  Preparing metadata (setup.py) ... done
  Created wheel for autograd-gamma: filename=autograd_gamma-0.5.0-py3-none-any.whl size=4031 sha256=77dd005fb64221d89755c50e9a315123eab507d4fd077e494599b3b283dfc9a7
  Stored in directory: /root/.cache/pip/wheels/6b/b5/e0/4c79e15c0b5f2c15ecf613c720bb20daab20a666eb67135155
Successfully built autograd-gamma
Processing /kaggle/input/pip-install-lifelines/interface_meta-1.3.0-py3-none-any.whl
Processing /kaggle/input/pip-install-lifelines/formulai

In [2]:
import numpy as np
import pandas as pd
import torch
from lifelines import KaplanMeierFitter, NelsonAalenFitter
from sklearn.impute import SimpleImputer # 欠損値の補完
from sklearn.preprocessing import LabelEncoder, StandardScaler # エンコードと標準化
from torch.utils.data import TensorDataset

# efs, efs_timeを生存関数に変換
def transform_target(train):
    kmf = KaplanMeierFitter() # イベントの生存確率推定
    # efs_timeとefsを用いて生存関数フィッティング
    kmf.fit(durations=train['efs_time'], event_observed=train['efs']) 
    # efs_timeにおける生存確率
    train['y'] = kmf.survival_function_at_times(train['efs_time']).values

    naf = NelsonAalenFitter() # 累積ハザード関数推定
    # efs_timeとefsを用いて生存関数フィッティング
    naf.fit(durations=train['efs_time'], event_observed=train['efs'])
    # efs_timeにおける生存確率
    train['y'] = naf.cumulative_hazard_at_times(train['efs_time']).values
    train['y'] = train['y'] * 1
    train['y'] = np.log(1 + train.efs_time) # スケール縮小
    return train

# df: データセット
# cat_cols: カテゴリ列のリストとして指定
# transformers: すでに学習されたカテゴリ変数変換エンコーダー
def get_X_cat(df, cat_cols, transformers=None):
    # カテゴリカル列に対してラベルエンコードを適用
    if transformers is None:
        transformers = [LabelEncoder().fit(df[col]) for col in cat_cols]

    # le.transformでラベルエンコード適用。T(転置)で変数名が列、サンプルが行になる
    return transformers, np.array(
        [transformer.transform(df[col]) for col, transformer in zip(cat_cols, transformers)]
    ).T

# カテゴリ列と数値列に分類する
def get_feature_types(train):
    categorical_cols = [col for i, col in enumerate(train.columns) if ((train[col].dtype == "object") |\
                                                                       (2 < train[col].nunique() < 25))]
    RMV = ["ID", "efs", "efs_time", "y"] # 除外する列名
    FEATURES = [c for c in train.columns if not c in RMV]
    print(f"There are {len(FEATURES)} FEATURES: {FEATURES}")
    # 数値列
    numerical = [i for i in FEATURES if i not in categorical_cols]
    return categorical_cols, numerical

# 訓練データ、検証データのカテゴリ列、数値列、ラベルエンコードを返す
def get_categoricals(train, val):
    categorical_cols, numerical = get_feature_types(train)
    remove = [] # 削除する列名
    for col in categorical_cols:
        if train[col].nunique() == 1: # ユニーク数が1の場合は削除
            remove.append(col)
        # valの列に訓練データが無い場合、nanに置き換える
        ind = ~val[col].isin(train[col]) # trainに無い列がvalにある場合
        if ind.any():
            val.loc[ind, col] = np.nan
    # colが除外列で無い場合
    categorical_cols = [col for col in categorical_cols if col not in remove]
    # 訓練データに関してラベルエンコードとカテゴリ列を取得
    transformers, X_cat_train = get_X_cat(train, categorical_cols)
    # 検証データに関してカテゴリ列を取得
    _, X_cat_val = get_X_cat(val, categorical_cols, transformers)
    return X_cat_train, X_cat_val, numerical, transformers

# データセットのテンソル化、データローダーの作成
def init_dl(X_cat, X_num, df, training=False):
    ds_train = TensorDataset(
        torch.tensor(X_cat, dtype=torch.long), # カテゴリdf
        torch.tensor(X_num, dtype=torch.float32), # 数値df
        torch.tensor(df.efs_time.values, dtype=torch.float32).log(),
        torch.tensor(df.efs.values, dtype=torch.long)
    )
    bs = 2048
    # if not training:
    #     bs = 2048 * 8
    dl_train = torch.utils.data.DataLoader(ds_train, batch_size=bs, pin_memory=True, shuffle=training)
    return dl_train

# 訓練、検証データの取得
def preprocess_data(train, val):
    # カテゴリデータ(訓練、検証)、数値列、ラベルエンコード取得
    X_cat_train, X_cat_val, numerical, transformers = get_categoricals(train, val)
    scaler = StandardScaler() # 標準化
    imp = SimpleImputer(missing_values=np.nan, strategy='mean', add_indicator=True) # 平均で補完
    # 標準化を実施
    X_num_train = imp.fit_transform(train[numerical])
    X_num_train = scaler.fit_transform(X_num_train)
    X_num_val = imp.transform(val[numerical])
    X_num_val = scaler.transform(X_num_val)
    # データローダー作成
    dl_train = init_dl(X_cat_train, X_num_train, train, training=True)
    dl_val = init_dl(X_cat_val, X_num_val, val)
    return X_cat_val, X_num_train, X_num_val, dl_train, dl_val, transformers

In [3]:
# 特徴量の追加
def add_features(df):
    
    # 性一致の列を追加
    sex_match = df.sex_match.astype(str) # 文字列結合
    sex_match = sex_match.str.split("-").str[0] == sex_match.str.split("-").str[1] # ハイフン取り除き一致確認
    df['sex_match_bool'] = sex_match # 列追加
    df.loc[df.sex_match.isna(), 'sex_match_bool'] = np.nan # 欠損はnanで埋める

    # 発病が16歳以上かどうか
    df['big_age'] = df.age_at_hct > 16

    # 0.044歳は異常値として加える
    df['strange_age'] = df.age_at_hct == 0.044
    
    # 手術を受けた年が2019の場合2020とする
    df.loc[df.year_hct == 2019, 'year_hct'] = 2020

    # 遺伝子数とその詳細の数字を加える
    df['is_cyto_score_same'] = (df['cyto_score'] == df['cyto_score_detail']).astype(int)

    # 年齢を等間隔に分ける
    df['age_bin'] = pd.cut(df.age_at_hct, [0, 0.0441, 16, 30, 50, 100])

    # 発病年齢 / ドナー年齢
    df['age_ts'] = df.age_at_hct / df.donor_age
    return df

In [4]:
# 訓練、テストデータの読み込み
def load_data():
    test = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/test.csv")
    # test = add_features(test)
    print("Test shape:", test.shape)
    train = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/train.csv")
    # train = add_features(train)
    print("Train shape:", train.shape)
    return test, train

In [5]:
import functools
from typing import List
import pytorch_lightning as pl
import numpy as np
import torch
from lifelines.utils import concordance_index
from pytorch_lightning.cli import ReduceLROnPlateau
from pytorch_tabular.models.common.layers import ODST
from torch import nn
from pytorch_lightning.utilities import grad_norm

class CatEmbeddings(nn.Module):
    def __init__(
        self,
        projection_dim: int,# 最終埋め込みベクトル次元数
        categorical_cardinality: List[int],#各カテゴリのユニーク数
        embedding_dim: int # 埋め込みベクトル次元数
    ):
        super(CatEmbeddings, self).__init__()
        # 各カテゴリのユニーク数　→　埋め込みベクトル次元数に圧縮
        self.embeddings = nn.ModuleList([nn.Embedding(cardinality, embedding_dim)
            for cardinality in categorical_cardinality
        ])
        # 次元圧縮
        self.projection = nn.Sequential(
            nn.Linear(embedding_dim * len(categorical_cardinality), projection_dim),
            nn.GELU(),
            nn.Linear(projection_dim, projection_dim)
        )
    # x_cat:カテゴリ変数
    def forward(self, x_cat):
        # 埋め込み次元圧縮
        x_cat = [embedding(x_cat[:, i]) for i, embedding in enumerate(self.embeddings)]
        # リストを列方向に結合しdfとする
        x_cat = torch.cat(x_cat, dim=1)
        return self.projection(x_cat) # MLP


class NN(nn.Module):
    def __init__(
            self,
            continuous_dim: int, # 数値列の次元数
            categorical_cardinality: List[int],# カテゴリ列ユニーク数
            embedding_dim: int, # 埋め込み次元数
            projection_dim: int, # 最終埋め込み次元数
            hidden_dim: int, # 隠れ層の次元数
            dropout: float = 0 # ドロップアウト確率
    ):
        super(NN, self).__init__()
        self.embeddings = CatEmbeddings(projection_dim, categorical_cardinality, embedding_dim)
        self.mlp = nn.Sequential(
            ODST(projection_dim + continuous_dim, hidden_dim), # 表形式のPytorch
            nn.BatchNorm1d(hidden_dim), # バッチ正規化
            nn.Dropout(dropout) # ドロップアウト
        )
        self.out = nn.Linear(hidden_dim, 1) # 最終
        self.dropout = nn.Dropout(dropout) # ドロップアウト

        # 初期重み
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_normal_(m.weight)
                nn.init.zeros_(m.bias)

    def forward(self, x_cat, x_cont):
        x = self.embeddings(x_cat) # 次元数削減
        x = torch.cat([x, x_cont], dim=1) # カテゴリ列と数値列の結合
        x = self.dropout(x) # ドロップアウト
        x = self.mlp(x) # 全結合層
        return self.out(x), x

# 関数の戻り値をキャッシュ
@functools.lru_cache
def combinations(N):
    ind = torch.arange(N) # 0からN-1までの整数
    # 2つの要素の組み合わせすべて
    comb = torch.combinations(ind, r=2)
    return comb.cuda()

# 生存分析を行うモデル
class LitNN(pl.LightningModule):
    def __init__(
            self,
            continuous_dim: int, # 数値列の次元数
            categorical_cardinality: List[int],# カテゴリ変数のユニーク数
            embedding_dim: int, # 埋め込み次元数
            projection_dim: int, # プロジェクション次元数
            hidden_dim: int,
            lr: float = 1e-3,
            dropout: float = 0.2,
            weight_decay: float = 1e-3,
            aux_weight: float = 0.1, # 補助損失の重み
            margin: float = 0.5, # 損失関数のマージンパラメータ
            race_index: int = 0 # 人種カテゴリインデックス
    ):
        # 継承
        super(LitNN, self).__init__()
        self.save_hyperparameters()
        self.model = NN(
            continuous_dim=self.hparams.continuous_dim,
            categorical_cardinality=self.hparams.categorical_cardinality,
            embedding_dim=self.hparams.embedding_dim,
            projection_dim=self.hparams.projection_dim,
            hidden_dim=self.hparams.hidden_dim,
            dropout=self.hparams.dropout
        )
        self.targets = []
        self.aux_cls = nn.Sequential(
            nn.Linear(self.hparams.hidden_dim, self.hparams.hidden_dim // 3),
            nn.GELU(),
            nn.Linear(self.hparams.hidden_dim // 3, 1)
        )

    # L2ノルム計算。ログ記録。
    def on_before_optimizer_step(self, optimizer):
        norms = grad_norm(self.model, norm_type=2)
        self.log_dict(norms)

    # 前方
    def forward(self, x_cat, x_cont):
        x, emb = self.model(x_cat, x_cont)
        return x.squeeze(1), emb

    # 訓練ステップ
    def training_step(self, batch, batch_idx):
        x_cat, x_cont, y, efs = batch
        # 予測、埋め込みベクトル計算
        y_hat, emb = self(x_cat, x_cont)
        aux_pred = self.aux_cls(emb).squeeze(1)
        loss, race_loss = self.get_full_loss(efs, x_cat, y, y_hat)
        aux_loss = nn.functional.mse_loss(aux_pred, y, reduction='none')
        aux_mask = efs == 1
        aux_loss = (aux_loss * aux_mask).sum() / aux_mask.sum()
        self.log("train_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        self.log("race_loss", race_loss, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        self.log("aux_loss", aux_loss, on_epoch=True, prog_bar=True, logger=True, on_step=False)
        return loss + aux_loss * self.hparams.aux_weight

    # 基本的な損失 + 人種による損失
    def get_full_loss(self, efs, x_cat, y, y_hat):
        loss = self.calc_loss(y, y_hat, efs)
        race_loss = self.get_race_losses(efs, x_cat, y, y_hat)
        loss += 0.1 * race_loss
        return loss, race_loss

    # 人種に対して個別に損失計算。損失の標準偏差を返す
    def get_race_losses(self, efs, x_cat, y, y_hat):
        races = torch.unique(x_cat[:, self.hparams.race_index])
        race_losses = []
        for race in races:
            ind = x_cat[:, self.hparams.race_index] == race
            race_losses.append(self.calc_loss(y[ind], y_hat[ind], efs[ind]))
        race_loss = sum(race_losses) / len(race_losses)
        races_loss_std = sum((r - race_loss)**2 for r in race_losses) / len(race_losses)
        return torch.sqrt(races_loss_std)

    # 生存分析とランキング学習に関連する損失計算
    def calc_loss(self, y, y_hat, efs):
        N = y.shape[0]
        comb = combinations(N)
        comb = comb[(efs[comb[:, 0]] == 1) | (efs[comb[:, 1]] == 1)]
        pred_left = y_hat[comb[:, 0]]
        pred_right = y_hat[comb[:, 1]]
        y_left = y[comb[:, 0]]
        y_right = y[comb[:, 1]]
        y = 2 * (y_left > y_right).int() - 1
        loss = nn.functional.relu(-y * (pred_left - pred_right) + self.hparams.margin)
        mask = self.get_mask(comb, efs, y_left, y_right)
        loss = (loss.double() * (mask.double())).sum() / mask.sum()
        return loss

    # comb:予測ペアのインデックスを含むテンソル
    # y_left = comb[:,0]
    # y_right = comb[:,1]
    def get_mask(self, comb, efs, y_left, y_right):
        # mask1 = (efs[comb[:, 0]] == 1) | (efs[comb[:, 1]] == 1)
        # 左が長生き
        left_outlived = y_left >= y_right
        # 左が死亡、右が未
        left_1_right_0 = (efs[comb[:, 0]] == 1) & (efs[comb[:, 1]] == 0)
        # 左が長生き、左が死、右が未
        mask2 = (left_outlived & left_1_right_0)
        # 右が長生き
        right_outlived = y_right >= y_left
        # 右が死亡、左が未
        right_1_left_0 = (efs[comb[:, 1]] == 1) & (efs[comb[:, 0]] == 0)
        mask2 |= (right_outlived & right_1_left_0)
        mask2 = ~mask2
        mask = mask2
        return mask

    # 検証ステップの損失計算
    def validation_step(self, batch, batch_idx):
        x_cat, x_cont, y, efs = batch
        y_hat, emb = self(x_cat, x_cont)
        loss, race_loss = self.get_full_loss(efs, x_cat, y, y_hat)
        self.targets.append([y, y_hat.detach(), efs, x_cat[:, self.hparams.race_index]])
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        return loss

    # エポックごとの検証結果を出力(cindex, カスタムメトリック)
    def on_validation_epoch_end(self):
        cindex, metric = self._calc_cindex()
        self.log("cindex", metric, on_epoch=True, prog_bar=True, logger=True)
        self.log("cindex_simple", cindex, on_epoch=True, prog_bar=True, logger=True)
        self.targets.clear()

    # C-indexとカスタムC-indexを返す
    def _calc_cindex(self):
        y = torch.cat([t[0] for t in self.targets]).cpu().numpy() # 実際の値
        y_hat = torch.cat([t[1] for t in self.targets]).cpu().numpy() # 予測値
        efs = torch.cat([t[2] for t in self.targets]).cpu().numpy() # フラグ
        races = torch.cat([t[3] for t in self.targets]).cpu().numpy() # 人種
        metric = self._metric(efs, races, y, y_hat)
        cindex = concordance_index(y, y_hat, efs)
        return cindex, metric

    # C-indexを修正
    def _metric(self, efs, races, y, y_hat):
        metric_list = []
        # 各人種ごとにC-indexを計算する
        for race in np.unique(races):
            y_ = y[races == race]
            y_hat_ = y_hat[races == race]
            efs_ = efs[races == race]
            metric_list.append(concordance_index(y_, y_hat_, efs_))
        metric = float(np.mean(metric_list) - np.sqrt(np.var(metric_list)))
        return metric

    # 損失計算
    def test_step(self, batch, batch_idx):
        # バッチデータ
        x_cat, x_cont, y, efs = batch
        # y_hat:予測値、emb:埋め込みベクトル
        y_hat, emb = self(x_cat, x_cont)
        # 損失計算
        loss, race_loss = self.get_full_loss(efs, x_cat, y, y_hat)
        # モデルの出力とターゲットを追加する
        self.targets.append([y, y_hat.detach(), efs, x_cat[:, self.hparams.race_index]])
        self.log("test_loss", loss)
        return loss

    # 評価指標のログ記録
    def on_test_epoch_end(self) -> None:
        cindex, metric = self._calc_cindex()
        self.log("test_cindex", metric, on_epoch=True, prog_bar=True, logger=True)
        self.log("test_cindex_simple", cindex, on_epoch=True, prog_bar=True, logger=True)
        self.targets.clear()

    # 最適化設定
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.hparams.lr, \
                                     weight_decay=self.hparams.weight_decay)
        scheduler_config = {
            # コサイン類似の学習率減衰スケジューラ
            "scheduler": torch.optim.lr_scheduler.CosineAnnealingLR(
                optimizer,
                T_max=45, # 学習率減衰までのエポック数
                eta_min=6e-3 # 学習率の最小値
            ),
            "interval": "epoch",
            "frequency": 1,
            "strict": False,
        }

        return {"optimizer": optimizer, "lr_scheduler": scheduler_config}



In [6]:
import json
import pytorch_lightning as pl
import numpy as np, pandas as pd
import matplotlib.pyplot as plt
import torch
from pytorch_lightning.callbacks import LearningRateMonitor, TQDMProgressBar
from pytorch_lightning.callbacks import StochasticWeightAveraging
from sklearn.model_selection import StratifiedKFold

pl.seed_everything(42)

def main(hparams):
    test, train_original = load_data()
    test['efs_time'] = 1
    test['efs'] = 1
    test_pred = np.zeros(test.shape[0])
    categorical_cols, numerical = get_feature_types(train_original)
    # 層状K分割
    kf = StratifiedKFold(n_splits=5, shuffle=True, )
    for i, (train_index, test_index) in enumerate(
        kf.split(
            train_original, train_original.race_group.astype(str) + (train_original.age_at_hct == 0.044).astype(str)
        )
    ):
        tt = train_original.copy()
        train = tt.iloc[train_index]
        val = tt.iloc[test_index]
        X_cat_val, X_num_train, X_num_val, dl_train, dl_val, transformers = preprocess_data(train, val)
        model = train_final(X_num_train, dl_train, dl_val, transformers, categorical_cols=categorical_cols)
        # Create submission
        train = tt.iloc[train_index]
        X_cat_val, X_num_train, X_num_val, dl_train, dl_val, transformers = preprocess_data(train, test)
        pred, _ = model.cuda().eval()(
            torch.tensor(X_cat_val, dtype=torch.long).cuda(),
            torch.tensor(X_num_val, dtype=torch.float32).cuda()
        )
        test_pred += pred.detach().cpu().numpy()
        
    subm_data = pd.read_csv("/kaggle/input/equity-post-HCT-survival-predictions/sample_submission.csv")
    subm_data['prediction'] = -test_pred
    subm_data.to_csv('submission.csv', index=False)
    
    display(subm_data.head())
    return 

# 全結合層での計算
def train_final(X_num_train, dl_train, dl_val, transformers, hparams=None, categorical_cols=None):
    if hparams is None:
        hparams = {
            "embedding_dim": 16,
            "projection_dim": 112,
            "hidden_dim": 56,
            "lr": 0.06464861983337984,
            "dropout": 0.05463240181423116,
            "aux_weight": 0.26545778308743806,
            "margin": 0.2588153271003354,
            "weight_decay": 0.0002773544957610778
        }
    model = LitNN(
        continuous_dim=X_num_train.shape[1],
        categorical_cardinality=[len(t.classes_) for t in transformers],
        race_index=categorical_cols.index("race_group"),
        **hparams
    )
    checkpoint_callback = pl.callbacks.ModelCheckpoint(monitor="val_loss", save_top_k=1)
    trainer = pl.Trainer(
        accelerator='cuda',
        max_epochs=60,
        callbacks=[
            checkpoint_callback,
            LearningRateMonitor(logging_interval='epoch'),
            TQDMProgressBar(),
            StochasticWeightAveraging(swa_lrs=1e-5, swa_epoch_start=45, annealing_epochs=15)
        ],
    )
    trainer.fit(model, dl_train)
    trainer.test(model, dl_val)
    return model.eval()

hparams = None
res = main(hparams)
print(res)

Test shape: (3, 58)
Train shape: (28800, 60)
There are 57 FEATURES: ['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia', 'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status', 'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6', 'hla_match_c_low', 'rituximab', 'hla_match_drb1_low', 'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe', 'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer', 'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue', 'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score', 'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_dose', 'hla_low_res_8', 'cardiac', 'hla_match_drb1_high', 'pulm_moderate', 'hla_low_res_

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/configuration_validator.py:70: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=3` in the `DataLoader` to improve performance.
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (12) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:384: `ModelCheckpoint(monitor='val_loss')` could not find the monitored key in the returned metrics: ['lr-Adam', 'train_loss', 'train_loss_step', 'grad_2.0_norm/embeddings.embeddings.0.weight', 'grad_2.0_norm/embeddings.embeddings.1.weight', 'grad_2.0_norm/embeddings.embeddings.2.weight', 'grad_2.0_norm/embeddings.embeddings.3.weight', 'grad_2.0_norm/embeddings.embeddings.4.weight', 'grad_2.0_norm/embeddings.embeddings.5.weight', 'grad_2.0_norm/embeddings.embeddings.6.weight', 'grad_2.0_norm/embeddings.embeddings.7.weight', 'grad_2.0_norm/embeddings.embeddings.8.weight', 'grad_2.0_norm/embeddings.embeddings.9.weight', 'grad_2.0_norm/embeddings.embeddings.10.weight', 'grad_2.0_norm/embeddings.embeddings.11.weight', 'grad_2.0_norm/embeddings.embeddings.12.weight', 'grad_2.0_norm/embeddings.embeddings.13.weight', 'grad_2.0_norm/embeddings.embeddings.14.weight', 'grad_2.0_norm/embeddings.embeddings.15.w

Testing: |          | 0/? [00:00<?, ?it/s]

...

Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        test_cindex        │    0.6698457598686218     │
│    test_cindex_simple     │    0.6787081956863403     │
│         test_loss         │    0.19026513342459392    │
└───────────────────────────┴───────────────────────────┘

There are 57 FEATURES: ['dri_score', 'psych_disturb', 'cyto_score', 'diabetes', 'hla_match_c_high', 'hla_high_res_8', 'tbi_status', 'arrhythmia', 'hla_low_res_6', 'graft_type', 'vent_hist', 'renal_issue', 'pulm_severe', 'prim_disease_hct', 'hla_high_res_6', 'cmv_status', 'hla_high_res_10', 'hla_match_dqb1_high', 'tce_imm_match', 'hla_nmdp_6', 'hla_match_c_low', 'rituximab', 'hla_match_drb1_low', 'hla_match_dqb1_low', 'prod_type', 'cyto_score_detail', 'conditioning_intensity', 'ethnicity', 'year_hct', 'obesity', 'mrd_hct', 'in_vivo_tcd', 'tce_match', 'hla_match_a_high', 'hepatic_severe', 'donor_age', 'prior_tumor', 'hla_match_b_low', 'peptic_ulcer', 'age_at_hct', 'hla_match_a_low', 'gvhd_proph', 'rheum_issue', 'sex_match', 'hla_match_b_high', 'race_group', 'comorbidity_score', 'karnofsky_score', 'hepatic_mild', 'tce_div_match', 'donor_related', 'melphalan_dose', 'hla_low_res_8', 'cardiac', 'hla_match_drb1_high', 'pulm_moderate', 'hla_low_res_10']


,ID,prediction
0,28800,-2.186416
1,28801,0.231354
2,28802,-2.996465


None
